##### Copyright 2020 The TensorFlow Quantum Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[link text](https://)# Binary classification of quantum states

Initial Tutorial Author : Antonio J. Martinez

Initial Tutorial Contributors : Masoud Mohseni

Initial Tutorial Created : 2020-Feb-14

Initial Tutorial Last updated : 2020-Feb-29

---

Current Experiment Author : Anneliese Brei

Current Experiment Created : 2022-Jan-3

Current Experiment Last updated : 2022-Jan-3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tensorflow/quantum/blob/research/binary_classifier/binary_classifier.ipynb)

An elementary learning task is [binary classification](https://en.wikipedia.org/wiki/Binary_classification), a supervised task in which the learner is to distinguish which of two classes a given datapoint has been drawn from.  Here, using ideas from the paper [Universal discriminative quantum neural networks](https://arxiv.org/abs/1805.08654) in the one-qubit setting, we train a hybrid quantum-classical neural network to distinguish between quantum data sources.

## Import dependencies

In [ ]:
!pip install --upgrade tensorflow
!pip install qutip

     |████████████████████████████████| 462 kB 4.2 MB/s 
     |████████████████████████████████| 14.8 MB 238 kB/s 


In [ ]:
!pip install tensorflow-quantum

     |████████████████████████████████| 10.5 MB 4.5 MB/s 
     |████████████████████████████████| 90 kB 7.3 MB/s 
     |████████████████████████████████| 1.6 MB 7.1 MB/s 
     |████████████████████████████████| 6.1 MB 29.7 MB/s 
     |████████████████████████████████| 100 kB 7.4 MB/s 
     |████████████████████████████████| 90 kB 7.7 MB/s 
     |████████████████████████████████| 437 kB 43.3 MB/s 
     |████████████████████████████████| 93 kB 1.2 MB/s 
     |████████████████████████████████| 93 kB 882 kB/s 
     |████████████████████████████████| 93 kB 643 kB/s 
     |████████████████████████████████| 93 kB 1.1 MB/s 
     |████████████████████████████████| 93 kB 858 kB/s 
     |████████████████████████████████| 93 kB 378 kB/s 
     |████████████████████████████████| 93 kB 1.1 MB/s 
     |████████████████████████████████| 93 kB 472 kB/s 
     |████████████████████████████████| 92 kB 510 kB/s 
     |████████████████████████████████| 93 kB 723 kB/s 
     |████████████████████████████████| 

In [ ]:
import cirq
import numpy as np
import qutip
import random
import sympy
import tensorflow as tf
import tensorflow_quantum as tfq

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## Quantum dataset
For our quantum dataset, you will generate two blobs on the surface of the Bloch sphere.  The task will be to learn a model to distinguish members of these blobs.  To do this, you first select two axes in the X-Z plane of the block sphere, then select random points uniformly distributed around them:

In [ ]:
def generate_dataset(qubit, theta_a, theta_b, num_samples):
  """Generate a dataset of points on `qubit` near the two given angles; labels
  for the two clusters use a one-hot encoding.
  """
  q_data = []
  bloch = {"a": [[], [], []], "b": [[], [], []]}
  labels = []
  blob_size = abs(theta_a - theta_b) / 5
  for _ in range(num_samples):
    coin = random.random()
    spread_x = np.random.uniform(-blob_size, blob_size)
    spread_y = np.random.uniform(-blob_size, blob_size)
    if coin < 0.5:
      label = [1, 0]
      angle = theta_a + spread_y
      source = "a"
    else:
      label = [0, 1]
      angle = theta_b + spread_y
      source = "b"
    labels.append(label)
    q_data.append(cirq.Circuit(cirq.ry(-angle)(qubit), cirq.rx(-spread_x)(qubit)))
    bloch[source][0].append(np.cos(angle))
    bloch[source][1].append(np.sin(angle)*np.sin(spread_x))
    bloch[source][2].append(np.sin(angle)*np.cos(spread_x))
  return tfq.convert_to_tensor(q_data), np.array(labels), bloch

In [ ]:
def build_model(theta_a, theta_b):

  qubit = cirq.GridQubit(0, 0)

  # Build the quantum model layer 1
  theta = sympy.Symbol('theta')
  q_model = cirq.Circuit(cirq.ry(theta)(qubit))
  q_data_input = tf.keras.Input(
      shape=(), dtype=tf.dtypes.string)
  expectation = tfq.layers.PQC(q_model, cirq.Z(qubit))
  expectation_output = expectation(q_data_input)

  # Attach the classical SoftMax classifier
  classifier = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
  classifier_output = classifier(expectation_output)
  model = tf.keras.Model(inputs=q_data_input, outputs=classifier_output)

  # Standard compilation for classification
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
  
  return model, qubit

In [ ]:
# Constants
theta_a = 1
theta_b = 4

num_samples = 5000 # Number datapoints
iterations = 10
num_epochs = 5

## 1-Node Framework
Implement Binary Classification in a federated learning framework using 2 separate models run by 2 different simulators. Model 0 represents the global model. Model 1 is a client. The parameters are averaged and used to update the global model.

In [ ]:
def run_1_node():

  model0, _ = build_model(theta_a, theta_b) # global
  model1, _ = build_model(theta_a, theta_b) # client

  loss = []       # Loss value from each iteration
  accuracy = []   # Accuracy % of each iteration

  for round in range(iterations):

    # Generate new local data for client
    qubit = cirq.GridQubit(0, 0)
    q_data1, labels1, _ = generate_dataset(qubit, theta_a, theta_b, num_samples) 

    # Train client 1 with (all) local data
    history1 = model1.fit(x=q_data1, y=labels1, epochs=num_epochs, verbose=0)

    weights0 = model0.get_weights()   # Extract global weights (parameters)
    weights1 = model1.get_weights()   # Extract client weights (parameters)

    # Average weights
    avg_weights = weights0
    for j in range(len(weights0)):
      avg_weights[j] = (weights0[j] + weights1[j]) / 2
    
    #print("Averaged weights: ", avg_weights)  

    # Update weights
    model0.set_weights(avg_weights)
    model1.set_weights(avg_weights)

    # Test updated global model for metrics
    qubit = cirq.GridQubit(0, 0)
    test_data, test_labels, test_bloch_p = generate_dataset(qubit, theta_a, theta_b, 500)
    test_results = model0.evaluate(test_data, test_labels, verbose=0)

    loss.append(test_results[0])        # Save loss value of each round
    accuracy.append(test_results[1])    # Save accuracy of each round

  return loss, accuracy

## 2-Node Framework
Implement Binary Classification in a federated learning framework using 3 client nodes and the global node models run by 3 different simulators. Model 0 represents the global model. Models 1 and 2 are clients. The parameters are averaged and used to update the global model.

In [ ]:
def run_2_node():
    
  model0, _ = build_model(theta_a, theta_b) #global
  model1, _ = build_model(theta_a, theta_b) #client1
  model2, _ = build_model(theta_a, theta_b) #client2

  loss = []       # Loss value from each iteration
  accuracy = []   # Accuracy % of each iteration

  for round in range(iterations):

    # Generate new local data for clients, D2 and D3
    qubit = cirq.GridQubit(0, 0)
    q_data, labels, _ = generate_dataset(qubit, theta_a, theta_b, num_samples) 
    
    q_data1 = q_data[:2500]
    q_data2 = q_data[2500:]
    labels1 = labels[:2500]
    labels2 = labels[2500:]

    # Train clients with local data
    history1 = model1.fit(x=q_data1, y=labels1, epochs=num_epochs, verbose=0)
    history2 = model2.fit(x=q_data2, y=labels2, epochs=num_epochs, verbose=0)

    weights0 = model0.get_weights()   # Extract global weights (parameters)
    weights1 = model1.get_weights()   # Extract client weights (parameters)
    weights2 = model2.get_weights()   # Extract client weights (parameters)

    # Average weights
    avg_weights = weights0
    for j in range(len(weights0)):
      avg_weights[j] = (weights0[j] + weights1[j] + weights2[j]) / 3 

    # Update weights
    model0.set_weights(avg_weights)
    model1.set_weights(avg_weights)
    model2.set_weights(avg_weights)

    # Test updated global model for metrics
    qubit = cirq.GridQubit(0, 0)
    test_data, test_labels, test_bloch_p = generate_dataset(qubit, theta_a, theta_b, 500)
    test_results = model0.evaluate(test_data,test_labels, verbose=0)

    loss.append(test_results[0])        # Save loss value of each round
    accuracy.append(test_results[1])    # Save accuracy of each round

  return loss, accuracy

## 5-Node Framework
Implement Binary Classification in a federated learning framework using 5 separate models run by 5 different simulators. Model 0 represents the global model. Models 1, 2, 3, 4 are clients. The parameters are averaged and used to update the global model.

In [ ]:
def run_5_node():

  # Create 5-node framework: 1 global model, 4 clients
  model0, _ = build_model(theta_a, theta_b) #global
  model1, _ = build_model(theta_a, theta_b) #client 1
  model2, _ = build_model(theta_a, theta_b) #client 2
  model3, _ = build_model(theta_a, theta_b) #client 3
  model4, _ = build_model(theta_a, theta_b) #client 4
  model5, _ = build_model(theta_a, theta_b) #client 5

  loss = []       # Loss value from each iteration
  accuracy = []   # Accuracy % of each iteration

  for round in range(iterations):

    # Generate new local data for clients, D1, D2, D3, D4
    qubit = cirq.GridQubit(0, 0)
    q_data, labels, _ = generate_dataset(qubit, theta_a, theta_b, num_samples)

    q_data1 = q_data[:1000]
    q_data2 = q_data[1000:2000]
    q_data3 = q_data[2000:3000]
    q_data4 = q_data[3000:4000]
    q_data5 = q_data[4000:]

    labels1 = labels[:1000]
    labels2 = labels[1000:2000]
    labels3 = labels[2000:3000]
    labels4 = labels[3000:4000]
    labels5 = labels[4000:] 

    # Train clients with local data
    history1 = model1.fit(x=q_data1, y=labels1, epochs=num_epochs, verbose=0)
    history2 = model2.fit(x=q_data2, y=labels2, epochs=num_epochs, verbose=0)
    history3 = model3.fit(x=q_data3, y=labels3, epochs=num_epochs, verbose=0)
    history4 = model4.fit(x=q_data4, y=labels4, epochs=num_epochs, verbose=0)
    history5 = model5.fit(x=q_data5, y=labels5, epochs=num_epochs, verbose=0)

    weights0 = model0.get_weights()   # Extract global weights (parameters)
    weights1 = model1.get_weights()   # Extract client weights (parameters)
    weights2 = model2.get_weights()   # Extract client weights (parameters)
    weights3 = model3.get_weights()   # Extract client weights (parameters)
    weights4 = model4.get_weights()   # Extract client weights (parameters)
    weights5 = model5.get_weights()   # Extract client weights (parameters)

    # Average weights
    avg_weights = weights0
    for j in range(len(weights0)):
      avg_weights[j] = (weights0[j] + weights1[j] + weights2[j] + weights3[j] + weights4[j] + weights5[j]) / 6

    # Update weights
    model0.set_weights(avg_weights)
    model1.set_weights(avg_weights)
    model2.set_weights(avg_weights)
    model3.set_weights(avg_weights)
    model4.set_weights(avg_weights)
    model5.set_weights(avg_weights)

    # Test updated global model for metrics
    qubit = cirq.GridQubit(0, 0)
    test_data, test_labels, test_bloch_p = generate_dataset(qubit, theta_a, theta_b, 500)
    test_results = model0.evaluate(test_data,test_labels, verbose=0)

    loss.append(test_results[0])        # Save loss value of each round
    accuracy.append(test_results[1])    # Save accuracy of each round

  return loss, accuracy

## 10-Node Framework
Implement Binary Classification in a federated learning framework using 10 clients. Model 0 represents the global model. Models 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 are clients. The parameters are averaged and used to update the global model.

In [ ]:
def run_10_node():

  model0, qubit0 = build_model(theta_a, theta_b) #global
  model1, qubit1 = build_model(theta_a, theta_b)
  model2, qubit2 = build_model(theta_a, theta_b)
  model3, qubit3 = build_model(theta_a, theta_b)
  model4, qubit4 = build_model(theta_a, theta_b)
  model5, qubit5 = build_model(theta_a, theta_b)
  model6, qubit6 = build_model(theta_a, theta_b)
  model7, qubit7 = build_model(theta_a, theta_b)
  model8, qubit8 = build_model(theta_a, theta_b)
  model9, qubit9 = build_model(theta_a, theta_b)
  model10, qubit10 = build_model(theta_a, theta_b)

  loss = []       # Loss value from each epoch
  accuracy = []   # Accuracy % of each epoch

  for round in range(iterations):

    # Generate new local data for clients, D1, D2, D3, D4
    qubit = cirq.GridQubit(0, 0)
    q_data, labels, bloch_p1 = generate_dataset(qubit, theta_a, theta_b, num_samples) 

    q_data1 = q_data[:500]
    q_data2 = q_data[500:1000]
    q_data3 = q_data[1000:1500]
    q_data4 = q_data[1500:2000]
    q_data5 = q_data[2000:2500]
    q_data6 = q_data[2500:3000]
    q_data7 = q_data[3000:3500]
    q_data8 = q_data[3500:4000]
    q_data9 = q_data[4000:4500]
    q_data10 = q_data[4500:5000]
    
    labels1 = labels[:500]
    labels2 = labels[500:1000]
    labels3 = labels[1000:1500]
    labels4 = labels[1500:2000]
    labels5 = labels[2000:2500] 
    labels6 = labels[2500:3000]
    labels7 = labels[3000:3500]
    labels8 = labels[3500:4000]
    labels9 = labels[4000:4500]
    labels10 = labels[4500:5000]

    # Train clients with local data
    history1 = model1.fit(x=q_data1, y=labels1, epochs=num_epochs, verbose=0)
    history2 = model2.fit(x=q_data2, y=labels2, epochs=num_epochs, verbose=0)
    history3 = model3.fit(x=q_data3, y=labels3, epochs=num_epochs, verbose=0)
    history4 = model4.fit(x=q_data4, y=labels4, epochs=num_epochs, verbose=0)
    history5 = model5.fit(x=q_data5, y=labels5, epochs=num_epochs, verbose=0)
    history6 = model6.fit(x=q_data6, y=labels6, epochs=num_epochs, verbose=0)
    history7 = model7.fit(x=q_data7, y=labels7, epochs=num_epochs, verbose=0)
    history8 = model8.fit(x=q_data8, y=labels8, epochs=num_epochs, verbose=0)
    history9 = model9.fit(x=q_data9, y=labels9, epochs=num_epochs, verbose=0)
    history10 = model10.fit(x=q_data10, y=labels10, epochs=num_epochs, verbose=0)

    weights0 = model0.get_weights()   # Extract global weights (parameters)
    weights1 = model1.get_weights()   # Extract client weights (parameters)
    weights2 = model2.get_weights()   # Extract client weights (parameters)
    weights3 = model3.get_weights()   # Extract client weights (parameters)
    weights4 = model4.get_weights()   # Extract client weights (parameters)
    weights5 = model5.get_weights()   # Extract global weights (parameters)
    weights6 = model6.get_weights()   # Extract client weights (parameters)
    weights7 = model7.get_weights()   # Extract client weights (parameters)
    weights8 = model8.get_weights()   # Extract client weights (parameters)
    weights9 = model9.get_weights()   # Extract client weights (parameters)
    weights10 = model10.get_weights()   # Extract client weights (parameters)

    # Average weights
    avg_weights = weights0
    for j in range(len(weights0)):
      avg_weights[j] = (weights0[j] + weights1[j] + weights2[j] + weights3[j] 
                        + weights4[j] + weights5[j] + weights6[j] + weights7[j] 
                        + weights8[j] + weights9[j] + weights10[j]) / 11

    # Update weights
    model0.set_weights(avg_weights)
    model1.set_weights(avg_weights)
    model2.set_weights(avg_weights)
    model3.set_weights(avg_weights)
    model4.set_weights(avg_weights)
    model5.set_weights(avg_weights)
    model6.set_weights(avg_weights)
    model7.set_weights(avg_weights)
    model8.set_weights(avg_weights)
    model9.set_weights(avg_weights)
    model10.set_weights(avg_weights)

    # Test updated global model for metrics
    qubit = cirq.GridQubit(0, 0)
    test_data, test_labels, test_bloch_p = generate_dataset(qubit, theta_a, theta_b, 500)
    test_results = model0.evaluate(test_data,test_labels, verbose=0)

    loss.append(test_results[0])        # Save loss value of each round
    accuracy.append(test_results[1])    # Save accuracy of each round

  return loss, accuracy

## Run frameworks

## 20-Node Framework
Implement Binary Classification in a federated learning framework using 20 clients. Model 0 represents the global model. Models 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20 are clients. The parameters are averaged and used to update the global model.

In [ ]:
def run_20_node():

  model0, qubit0 = build_model(theta_a, theta_b) #global
  model1, qubit1 = build_model(theta_a, theta_b)
  model2, qubit2 = build_model(theta_a, theta_b)
  model3, qubit3 = build_model(theta_a, theta_b)
  model4, qubit4 = build_model(theta_a, theta_b)
  model5, qubit5 = build_model(theta_a, theta_b)
  model6, qubit6 = build_model(theta_a, theta_b)
  model7, qubit7 = build_model(theta_a, theta_b)
  model8, qubit8 = build_model(theta_a, theta_b)
  model9, qubit9 = build_model(theta_a, theta_b)
  model10, qubit10 = build_model(theta_a, theta_b)
  model11, qubit11 = build_model(theta_a, theta_b)
  model12, qubit12 = build_model(theta_a, theta_b)
  model13, qubit13 = build_model(theta_a, theta_b)
  model14, qubit14 = build_model(theta_a, theta_b)
  model15, qubit15 = build_model(theta_a, theta_b)
  model16, qubit16 = build_model(theta_a, theta_b)
  model17, qubit17 = build_model(theta_a, theta_b)
  model18, qubit18 = build_model(theta_a, theta_b)
  model19, qubit19 = build_model(theta_a, theta_b)
  model20, qubit20 = build_model(theta_a, theta_b)

  loss = []       # Loss value from each epoch
  accuracy = []   # Accuracy % of each epoch

  for round in range(iterations):

    # Generate new local data for clients, D1, D2, D3, D4
    qubit = cirq.GridQubit(0, 0)
    q_data, labels, bloch_p = generate_dataset(qubit, theta_a, theta_b, num_samples) 

    q_data1 = q_data[:250]
    q_data2 = q_data[250:500]
    q_data3 = q_data[500:750]
    q_data4 = q_data[750:1000]
    q_data5 = q_data[1000:1250]
    q_data6 = q_data[1250:1500]
    q_data7 = q_data[1500:1750]
    q_data8 = q_data[1750:2000]
    q_data9 = q_data[2000:2250]
    q_data10 = q_data[2250:2500]
    q_data11 = q_data[2500:2750]
    q_data12 = q_data[2750:3000]
    q_data13 = q_data[3000:3250]
    q_data14 = q_data[3250:3500]
    q_data15 = q_data[3500:3750]
    q_data16 = q_data[3750:4000]
    q_data17 = q_data[4000:4250]
    q_data18 = q_data[4250:4500]
    q_data19 = q_data[4500:4750]
    q_data20 = q_data[4750:5000]
    
    labels1 = labels[:250]
    labels2 = labels[250:500]
    labels3 = labels[500:750]
    labels4 = labels[750:1000]
    labels5 = labels[1000:1250] 
    labels6 = labels[1250:1500]
    labels7 = labels[1500:1750]
    labels8 = labels[1750:2000]
    labels9 = labels[2000:2250]
    labels10 = labels[2250:2500]
    labels11 = labels[2500:2750]
    labels12 = labels[2750:3000]
    labels13 = labels[3000:3250]
    labels14 = labels[3250:3500]
    labels15 = labels[3500:3750] 
    labels16 = labels[3750:4000]
    labels17 = labels[4000:4250]
    labels18 = labels[4250:4500]
    labels19 = labels[4500:4750]
    labels20 = labels[4750:5000]

    # Train clients with local data
    history1 = model1.fit(x=q_data1, y=labels1, epochs=num_epochs, verbose=0)
    history2 = model2.fit(x=q_data2, y=labels2, epochs=num_epochs, verbose=0)
    history3 = model3.fit(x=q_data3, y=labels3, epochs=num_epochs, verbose=0)
    history4 = model4.fit(x=q_data4, y=labels4, epochs=num_epochs, verbose=0)
    history5 = model5.fit(x=q_data5, y=labels5, epochs=num_epochs, verbose=0)
    history6 = model6.fit(x=q_data6, y=labels6, epochs=num_epochs, verbose=0)
    history7 = model7.fit(x=q_data7, y=labels7, epochs=num_epochs, verbose=0)
    history8 = model8.fit(x=q_data8, y=labels8, epochs=num_epochs, verbose=0)
    history9 = model9.fit(x=q_data9, y=labels9, epochs=num_epochs, verbose=0)
    history10 = model10.fit(x=q_data10, y=labels10, epochs=num_epochs, verbose=0)
    history11 = model11.fit(x=q_data11, y=labels11, epochs=num_epochs, verbose=0)
    history12 = model12.fit(x=q_data12, y=labels12, epochs=num_epochs, verbose=0)
    history13 = model13.fit(x=q_data13, y=labels13, epochs=num_epochs, verbose=0)
    history14 = model14.fit(x=q_data14, y=labels14, epochs=num_epochs, verbose=0)
    history15 = model15.fit(x=q_data15, y=labels15, epochs=num_epochs, verbose=0)
    history16 = model16.fit(x=q_data16, y=labels16, epochs=num_epochs, verbose=0)
    history17 = model17.fit(x=q_data17, y=labels17, epochs=num_epochs, verbose=0)
    history18 = model18.fit(x=q_data18, y=labels18, epochs=num_epochs, verbose=0)
    history19 = model19.fit(x=q_data19, y=labels19, epochs=num_epochs, verbose=0)
    history20 = model20.fit(x=q_data20, y=labels20, epochs=num_epochs, verbose=0)

    # Extract global weights (parameters)
    weights0 = model0.get_weights()   
    weights1 = model1.get_weights()  
    weights2 = model2.get_weights()  
    weights3 = model3.get_weights()  
    weights4 = model4.get_weights()   
    weights5 = model5.get_weights() 
    weights6 = model6.get_weights()   
    weights7 = model7.get_weights()   
    weights8 = model8.get_weights()  
    weights9 = model9.get_weights()  
    weights10 = model10.get_weights()  
    weights11 = model11.get_weights()  
    weights12 = model12.get_weights()  
    weights13 = model13.get_weights()  
    weights14 = model14.get_weights()   
    weights15 = model15.get_weights() 
    weights16 = model16.get_weights()   
    weights17 = model17.get_weights()   
    weights18 = model18.get_weights()  
    weights19 = model19.get_weights()  
    weights20 = model20.get_weights()   

    # Average weights
    avg_weights = weights0
    for j in range(len(weights0)):
      avg_weights[j] = (weights0[j] + weights1[j] + weights2[j] + weights3[j] 
                        + weights4[j] + weights5[j] + weights6[j] + weights7[j] 
                        + weights8[j] + weights9[j] + weights10[j]
                        + weights11[j] + weights12[j] + weights13[j] 
                        + weights14[j] + weights15[j] + weights16[j] + weights17[j] 
                        + weights18[j] + weights19[j] + weights20[j]) / 21

    # Update weights
    model0.set_weights(avg_weights)
    model1.set_weights(avg_weights)
    model2.set_weights(avg_weights)
    model3.set_weights(avg_weights)
    model4.set_weights(avg_weights)
    model5.set_weights(avg_weights)
    model6.set_weights(avg_weights)
    model7.set_weights(avg_weights)
    model8.set_weights(avg_weights)
    model9.set_weights(avg_weights)
    model10.set_weights(avg_weights)
    model11.set_weights(avg_weights)
    model12.set_weights(avg_weights)
    model13.set_weights(avg_weights)
    model14.set_weights(avg_weights)
    model15.set_weights(avg_weights)
    model16.set_weights(avg_weights)
    model17.set_weights(avg_weights)
    model18.set_weights(avg_weights)
    model19.set_weights(avg_weights)
    model20.set_weights(avg_weights)

    # Test updated global model for metrics
    qubit = cirq.GridQubit(0, 0)
    test_data, test_labels, test_bloch_p = generate_dataset(qubit, theta_a, theta_b, 500)
    test_results = model0.evaluate(test_data,test_labels, verbose=0)

    loss.append(test_results[0])        # Save loss value of each round
    accuracy.append(test_results[1])    # Save accuracy of each round

  return loss, accuracy

In [ ]:
loss1_1, accuracy1_1 = run_1_node()
loss1_2, accuracy1_2 = run_1_node()
loss1_3, accuracy1_3 = run_1_node()
loss1_4, accuracy1_4 = run_1_node()
loss1_5, accuracy1_5 = run_1_node()

In [ ]:
loss1 = []
accuracy1 = []

# Calculate average loss
for i in range(len(loss1_1)):
  loss1.append((loss1_1[i] + loss1_2[i] + loss1_3[i] + loss1_4[i] + loss1_5[i]) / 5)

print('loss1_1 :', loss1_1)
print('loss1_2 :', loss1_2)
print('loss1_3 :', loss1_3)
print('loss1_4 :', loss1_4)
print('loss1_5 :', loss1_5)
print('loss1 : ', loss1)


# Calculate average accuracy
for i in range(len(accuracy1_1)):
  accuracy1.append((accuracy1_1[i] + accuracy1_2[i] + accuracy1_3[i] + accuracy1_4[i] + accuracy1_5[i]) / 5)

print('accuracy1_1 :', accuracy1_1)
print('accuracy1_2 :', accuracy1_2)
print('accuracy1_3 :', accuracy1_3)
print('accuracy1_4 :', accuracy1_4)
print('accuracy1_5 :', accuracy1_5)
print('accuracy1 : ', accuracy1)

loss1_1 : [2.15166974067688, 0.10122592002153397, 0.0013662538258358836, 0.0001621867559151724, 5.293066715239547e-05, 2.588459028629586e-05, 1.4876136447128374e-05, 9.11751703824848e-06, 5.991897069179686e-06, 4.2743454287119675e-06]
loss1_2 : [0.06663788110017776, 0.00395142612978816, 0.0005666977376677096, 0.00018395567894913256, 8.329839329235256e-05, 4.665280721383169e-05, 2.727819446590729e-05, 1.7137443137471564e-05, 1.0656955055310391e-05, 6.747438874299405e-06]
loss1_3 : [0.2619248330593109, 0.007000266574323177, 0.0006936350837349892, 0.00013865591608919203, 5.153041274752468e-05, 2.584010871942155e-05, 1.5552739569102414e-05, 9.896883966575842e-06, 5.799491646030219e-06, 3.752454176719766e-06]
loss1_4 : [1.5702266693115234, 0.08411718904972076, 0.0016238261014223099, 0.0001997999643208459, 6.14240561844781e-05, 2.9765922590740956e-05, 1.8538597942097113e-05, 1.132546276494395e-05, 6.799404218327254e-06, 4.474136403587181e-06]
loss1_5 : [0.02150203287601471, 0.001877846894785

In [ ]:
loss2_1, accuracy2_1 = run_2_node()
loss2_2, accuracy2_2 = run_2_node()
loss2_3, accuracy2_3 = run_2_node()
loss2_4, accuracy2_4 = run_2_node()
loss2_5, accuracy2_5 = run_2_node()

In [ ]:
loss2 = []
accuracy2 = []

# Calculate average loss
for i in range(len(loss2_1)):
  loss2.append((loss2_1[i] + loss2_2[i] + loss2_3[i] + loss2_4[i] + loss2_5[i]) / 5)

print('loss2_1 :', loss2_1)
print('loss2_2 :', loss2_2)
print('loss2_3 :', loss2_3)
print('loss2_4 :', loss2_4)
print('loss2_5 :', loss2_5)
print('loss2 : ', loss2)


# Calculate average accuracy
for i in range(len(accuracy2_1)):
  accuracy2.append((accuracy2_1[i] + accuracy2_2[i] + accuracy2_3[i] + accuracy2_4[i] + accuracy2_5[i]) / 5)

print('accuracy2_1 :', accuracy2_1)
print('accuracy2_2 :', accuracy2_2)
print('accuracy2_3 :', accuracy2_3)
print('accuracy2_4 :', accuracy2_4)
print('accuracy2_5 :', accuracy2_5)
print('accuracy2 : ', accuracy2)

loss2_1 : [0.6747549176216125, 0.01194025482982397, 0.0010815371060743928, 0.0003824167652055621, 0.00021161383483558893, 0.00013257841055747122, 9.779535321285948e-05, 6.539483729284257e-05, 5.146957118995488e-05, 3.577920142561197e-05]
loss2_2 : [4.671791076660156, 0.04101314768195152, 0.000896874931640923, 0.00029872028972022235, 0.0001666540774749592, 0.00011692709813360125, 8.820309449220076e-05, 6.45335967419669e-05, 4.986313433619216e-05, 4.0235008782474324e-05]
loss2_3 : [0.6473646759986877, 0.005730850622057915, 0.0005406226264312863, 0.0002220857422798872, 0.00012185538798803464, 8.204281039070338e-05, 5.324427911546081e-05, 3.8541715184692293e-05, 2.8433980332920328e-05, 2.0159597625024617e-05]
loss2_4 : [0.21470922231674194, 0.002951283473521471, 0.0005817413330078125, 0.0002637657162267715, 0.000150583335198462, 0.00010177010699408129, 7.507140253437683e-05, 5.2826286264462397e-05, 4.1036746551981196e-05, 3.144308357150294e-05]
loss2_5 : [0.056706950068473816, 0.0015516186

In [ ]:
loss5_1, accuracy5_1 = run_5_node()
loss5_2, accuracy5_2 = run_5_node()
loss5_3, accuracy5_3 = run_5_node()
loss5_4, accuracy5_4 = run_5_node()
loss5_5, accuracy5_5 = run_5_node()

KeyboardInterrupt: ignored

In [ ]:
loss5 = []
accuracy5 = []

# Calculate average loss
for i in range(len(loss5_1)):
  loss5.append((loss5_1[i] + loss5_2[i] + loss5_3[i] + loss5_4[i] + loss5_5[i]) / 5)

print('loss5_1 :', loss5_1)
print('loss5_2 :', loss5_2)
print('loss5_3 :', loss5_3)
print('loss5_4 :', loss5_4)
print('loss5_5 :', loss5_5)
print('loss5 : ', loss5)


# Calculate average accuracy
for i in range(len(accuracy5_1)):
  accuracy5.append((accuracy5_1[i] + accuracy5_2[i] + accuracy5_3[i] + accuracy5_4[i] + accuracy5_5[i]) / 5)

print('accuracy5_1 :', accuracy5_1)
print('accuracy5_2 :', accuracy5_2)
print('accuracy5_3 :', accuracy5_3)
print('accuracy5_4 :', accuracy5_4)
print('accuracy5_5 :', accuracy5_5)
print('accuracy5 : ', accuracy5)

In [ ]:
loss10_1, accuracy10_1 = run_10_node()
loss10_2, accuracy10_2 = run_10_node()
loss10_3, accuracy10_3 = run_10_node()
loss10_4, accuracy10_4 = run_10_node()
loss10_5, accuracy10_5 = run_10_node()

In [ ]:
loss10 = []
accuracy10 = []

# Calculate average loss
for i in range(len(loss10_1)):
  loss10.append((loss10_1[i] + loss10_2[i] + loss10_3[i] + loss10_4[i] + loss10_5[i]) / 5)

print('loss10_1 :', loss10_1)
print('loss10_2 :', loss10_2)
print('loss10_3 :', loss10_3)
print('loss10_4 :', loss10_4)
print('loss10_5 :', loss10_5)
print('loss10 : ', loss10)


# Calculate average accuracy
for i in range(len(accuracy10_1)):
  accuracy10.append((accuracy10_1[i] + accuracy10_2[i] + accuracy10_3[i] + accuracy10_4[i] + accuracy10_5[i]) / 5)

print('accuracy10_1 :', accuracy10_1)
print('accuracy10_2 :', accuracy10_2)
print('accuracy10_3 :', accuracy10_3)
print('accuracy10_4 :', accuracy10_4)
print('accuracy10_5 :', accuracy10_5)
print('accuracy10 : ', accuracy10)

In [ ]:
loss20_1, accuracy20_1 = run_20_node()
loss20_2, accuracy20_2 = run_20_node()
loss20_3, accuracy20_3 = run_20_node()
loss20_4, accuracy20_4 = run_20_node()
loss20_5, accuracy20_5 = run_20_node()

In [ ]:
loss20 = []
accuracy20 = []

# Calculate average loss
for i in range(len(loss20_1)):
  loss20.append((loss20_1[i] + loss20_2[i] + loss20_3[i] + loss20_4[i] + loss20_5[i]) / 5)

print('loss20_1 :', loss20_1)
print('loss20_2 :', loss20_2)
print('loss20_3 :', loss20_3)
print('loss20_4 :', loss20_4)
print('loss20_5 :', loss20_5)
print('loss20 : ', loss20)


# Calculate average accuracy
for i in range(len(accuracy20_1)):
  accuracy20.append((accuracy20_1[i] + accuracy20_2[i] + accuracy20_3[i] + accuracy20_4[i] + accuracy20_5[i]) / 5)

print('accuracy20_1 :', accuracy20_1)
print('accuracy20_2 :', accuracy20_2)
print('accuracy20_3 :', accuracy20_3)
print('accuracy20_4 :', accuracy20_4)
print('accuracy20_5 :', accuracy20_5)
print('accuracy20 : ', accuracy20)

In [ ]:
plt.plot(loss1, label="1 devices")
plt.plot(loss2, label="2 devices")
plt.plot(loss5, label="5 devices")
plt.plot(loss10, label="10 devices")
plt.plot(loss20, label="20 devices")
plt.xlabel("iterations")
plt.ylabel("loss value")
plt.legend()
plt.title("Loss on training data")
plt.show()

In [ ]:
plt.plot(accuracy1, label="1 device")
plt.plot(accuracy2, label="2 devices")
plt.plot(accuracy5, label="5 devices")
plt.plot(accuracy10, label="10 devices")
plt.plot(accuracy20, label="20 devices")
plt.xlabel("iterations")
plt.ylabel("accuracy")
plt.legend()
plt.title("accuracy on training data")
plt.show()

In [ ]:
print("Loss 1 :", loss1)
print("Loss 2 :", loss2)
print("Loss 5 :", loss5)
print("Loss 10 :", loss10)
print("Loss 20:", loss20)
print("-"*10)
print("Accuracy 1 : ", accuracy1)
print("Accuracy 2 : ", accuracy2)
print("Accuracy 5 : ", accuracy5)
print("Accuracy 10 : ", accuracy10)
print("Accuracy 20: ", accuracy20)